In [ ]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# FASE DE EXTRACCION DE DATOS

file_path = "data_ejercicio.xlsx"
excel_data = pd.ExcelFile(file_path)

print(excel_data.sheet_names)

In [ ]:
 Cargar las pestañas necesarias (ejemplo: "products" y "recommendations")
products_df = excel_data.parse("products")
recommendations_df = excel_data.parse("recommendations")
customers_last_month_df = excel_data.parse("customers_last_month")

In [ ]:
# Inspección básica de las tablas

print(products_df.head())
print(recommendations_df.head())
customers_last_month_df.head())

In [ ]:
# Verificar valores nulos

print(products_df.isnull().sum())
print(recommendations_df.isnull().sum())
print(customers_last_month_df.isnull().sum())

In [ ]:
# Limpiar datos si es necesario

products_df = products_df.dropna()
recommendations_df = recommendations_df.dropna()
customers_last_month_df = customers_last_month_df.dropna()



In [ ]:
# Crear una columna para validar la restricción

recommendations_df = recommendations_df.merge(
    products_df[['Product_desc', 'buyers_last_month']], 
    on='Product_desc', 
    how='left'
)

In [ ]:
# Calcular límites

recommendations_df['min_limit'] = recommendations_df['buyers_last_month'] * 0.75
recommendations_df['max_limit'] = recommendations_df['buyers_last_month'] * 2


In [ ]:
# Identificar violaciones

recommendations_df['violates'] = recommendations_df['Objetivo Anterior'].apply(
    lambda x: "low" if x < recommendations_df['min_limit'] else 
              "high" if x > recommendations_df['max_limit'] else "ok"
)

print(recommendations_df['violates'].value_counts())


In [ ]:
# Filtrar segmento premium

premium_products = products_df[products_df['segment'] == 'Premium']


In [ ]:
# Total de recomendaciones premium y generales

total_recommendations = recommendations_df['Objetivo Anterior'].sum()
premium_recommendations = recommendations_df[recommendations_df['Product_desc'].isin(premium_products['Product_desc'])]['Objetivo Anterior'].sum()


In [ ]:
# Validar porcentaje

premium_percentage = premium_recommendations / total_recommendations
print(f"Porcentaje actual Premium: {premium_percentage:.2%}")

In [ ]:
# Ajustar objetivo para Brahma Dorada

brahma = recommendations_df[recommendations_df['Product_desc'] == 'Brahma Dorada-1 Lts-Bottle-RET']
if brahma['Objetivo Anterior'].iloc[0] < 200:
    recommendations_df.loc[recommendations_df['Product_desc'] == 'Brahma Dorada-1 Lts-Bottle-RET', 'Objetivo Anterior'] = 200

In [ ]:
# Total de recomendaciones ajustadas

adjusted_total = recommendations_df['Objetivo Anterior'].sum()


In [ ]:
# Calculamos el factor de ajuste

scaling_factor = total_recommendations / adjusted_total


In [ ]:
# Aplicamos el ajuste

recommendations_df['Objetivo Nuevo'] = recommendations_df['Objetivo Anterior'] * scaling_factor


In [ ]:
# Redondear al entero más cercano

recommendations_df['Objetivo Nuevo'] = recommendations_df['Objetivo Nuevo'].round().astype(int)


In [ ]:
# Validar nuevo total

print(f"Nuevo total de recomendaciones: {recommendations_df['Objetivo Nuevo'].sum()}")


In [ ]:
output_path = "resultados_recomendaciones.xlsx"
recommendations_df.to_excel(output_path, index=False)
print(f"Resultados guardados en {output_path}")